In [1]:
import pandas as pd
from tqdm import tqdm

**Import all csvs and store into seppareta dataframes**

In [2]:
circuits = pd.read_csv('../data/raw/circuits.csv')
constructor_results = pd.read_csv('../data/raw/constructor_results.csv')
constructor_standings = pd.read_csv('../data/raw/constructor_standings.csv')
constructors = pd.read_csv('../data/raw/constructors.csv')
driver_standings = pd.read_csv('../data/raw/driver_standings.csv')
drivers = pd.read_csv('../data/raw/drivers.csv', parse_dates=[6])
lap_times = pd.read_csv('../data/raw/lap_times.csv')
pit_stops = pd.read_csv('../data/raw/pit_stops.csv')
qualifying = pd.read_csv('../data/raw/qualifying.csv')
races = pd.read_csv('../data/raw/races.csv', parse_dates=[5])
results = pd.read_csv('../data/raw/results.csv')
seasons = pd.read_csv('../data/raw/seasons.csv')
status = pd.read_csv('../data/raw/status.csv')

**Merge results with drivers**

In [3]:
result_with_dob = results.merge(drivers, left_on='driverId', right_on='driverId')
results_with_dob_date = result_with_dob.merge(races, on = 'raceId', how = 'left')
pd.to_datetime(results_with_dob_date['date'])
pd.to_datetime(results_with_dob_date['dob'])

0       1985-01-07
1       1985-01-07
2       1985-01-07
3       1985-01-07
4       1985-01-07
           ...    
25355   1999-03-02
25356   1999-03-02
25357   1999-03-02
25358   1999-03-02
25359   1999-03-02
Name: dob, Length: 25360, dtype: datetime64[ns]

**Calculate age at race in days (timedelta)**

In [4]:
results_with_dob_date['ageAtRace'] = results_with_dob_date['date'] - results_with_dob_date['dob']

**Calculate how many days in f1 in days (timedelta)**

In [5]:

date_of_debut = results_with_dob_date.groupby('driverId')['date'].min().to_frame()
date_of_debut = date_of_debut.merge(drivers, on = 'driverId', how = 'left').rename(columns={'date':'date_of_debut'})
date_of_debut = date_of_debut[['driverId','date_of_debut']]
date_of_debut['date_of_debut']
df = results_with_dob_date.merge(date_of_debut, on = 'driverId', how = 'left')
df['ageAtDebut'] = df['date_of_debut'] - df['dob']
df['yearsExperience'] = df['date'] - df['date_of_debut']

**Drop and rename columns**

In [6]:
df = df.drop(columns=['number_y', 'number_x', 'code', 'forename', 'surname', 'dob', 'url_x', 'time_y', 'url_y'])
df = df.rename(columns={"grid": "gridStart", "name": "circuitName", "date_of_debut" : "dateOfDebut", "points": "pointsGained"})

**Add is racing in home country feature**

In [7]:
df['racingAtHome'] = df.apply(lambda row: row.nationality in row.circuitName, axis=1)

**Convert ages from days to years**

In [8]:
df['ageAtRace'] = df.apply(lambda row: int(row.ageAtRace / pd.Timedelta('365 days')), axis=1)
df['ageAtDebut'] = df.apply(lambda row: int(row.ageAtDebut / pd.Timedelta('365 days')), axis=1)
df['yearsExperience'] = df.apply(lambda row: int(row.yearsExperience / pd.Timedelta('365 days')), axis=1)

**Merge with driver standings**

In [9]:
driver_standings = driver_standings.rename(columns={"points": "driverStandingsPoints", "position": "driverStandingsPosition", "wins" : "driverStandingsWins"})
driver_standings = driver_standings.drop(columns=['positionText'])
df = df.merge(driver_standings, on = ['raceId','driverId'], how = 'left')

**Get driver standings information before race**

In [10]:
df['driverStandingsPoints'] = df.groupby(['year','driverId'])['driverStandingsPoints'].shift(fill_value=0)
df['driverStandingsPosition'] = df.groupby(['year','driverId'])['driverStandingsPosition'].shift(fill_value=0)
df['driverStandingsWins'] = df.groupby(['year','driverId'])['driverStandingsWins'].shift(fill_value=0)
df['lastRaceRank'] = df.groupby(['year','driverId'])['rank'].shift()

**Merge with constructor standings**

In [11]:
constructor_standings = constructor_standings.rename(columns={"points": "constructorStandingsPoints", "position": "constructorStandingsPosition", "wins" : "constructorStandingsWins"})
constructor_standings = constructor_standings.drop(columns=['positionText'])
df = df.merge(constructor_standings, on = ['raceId','constructorId'], how = 'left')

**Get constructor standings information before race**

In [12]:
df['constructorStandingsPoints'] = df.groupby(['year','driverId'])['constructorStandingsPoints'].shift(fill_value=0)
df['constructorStandingsPosition'] = df.groupby(['year','driverId'])['constructorStandingsPosition'].shift(fill_value=0)
df['constructorStandingsWins'] = df.groupby(['year','driverId'])['constructorStandingsWins'].shift(fill_value=0)

**Get previous race grid and final race positions**

In [13]:
df['previousRaceGridStart'] = df.groupby(['year','driverId'])['gridStart'].shift()
df['previousRacePosition'] = df.groupby(['year','driverId'])['positionOrder'].shift()

**Add status for each row**

In [14]:
df = df.merge(status, on = ['statusId'], how = 'left')

**Get all historic features up until current row date**

In [15]:
for date in tqdm(df.date.unique()):
    #Races won by driver
    races_won_by_driver = df[df.date < date].groupby('driverId')['positionText'].apply(lambda x: (x=='1').sum()).reset_index(name='racesWon')
    races_won_by_driver['date'] = date
    dfaux = df[df['date'] == date].copy()
    dfaux['racesWon'] = 'to_delete'
    dfaux = dfaux.drop('racesWon', axis = 1)
    df.loc[df['date'] == date, 'racesWon'] = dfaux.merge(races_won_by_driver, on=['date','driverId'], how='left')['racesWon'].values
    
    #Number of races retired
    races_retired_by_driver = df[df.date < date].groupby('driverId')['positionText'].apply(lambda x: (x=='R').sum()).reset_index(name='racesRetired')
    races_retired_by_driver['date'] = date
    dfaux = df[df['date'] == date].copy()
    dfaux['racesRetired'] = 'to_delete'
    dfaux = dfaux.drop('racesRetired', axis = 1)
    df.loc[df['date'] == date, 'racesRetired'] = dfaux.merge(races_retired_by_driver, on=['date','driverId'], how='left')['racesRetired'].values

    #Number of races finished
    races_finished_by_driver = df[df.date < date].groupby('driverId')['status'].apply(lambda x: ((x=='Finished') | ("+" in x)).sum()).reset_index(name='racesFinished')
    races_finished_by_driver['date'] = date
    dfaux = df[df['date'] == date].copy()
    dfaux['racesFinished'] = 'to_delete'
    dfaux = dfaux.drop('racesFinished', axis = 1)
    df.loc[df['date'] == date, 'racesFinished'] = dfaux.merge(races_finished_by_driver, on=['date','driverId'], how='left')['racesFinished'].values

    #Number of pole positions
    pole_positions_by_driver = df[df.date < date].groupby('driverId')['gridStart'].apply(lambda x: (x==1).sum()).reset_index(name='polePositions')
    pole_positions_by_driver['date'] = date
    dfaux = df[df['date'] == date].copy()
    dfaux['polePositions'] = 'to_delete'
    dfaux = dfaux.drop('polePositions', axis = 1)
    df.loc[df['date'] == date, 'polePositions'] = dfaux.merge(pole_positions_by_driver, on=['date','driverId'], how='left')['polePositions'].values

    #Constructors races won
    races_won_by_constructor = df[df.date < date].groupby('constructorId')['positionText'].apply(lambda x: (x=='1').sum()).reset_index(name='racesWonByConstructor')
    races_won_by_constructor['date'] = date
    dfaux = df[df['date'] == date].copy()
    dfaux['racesWonByConstructor'] = 'to_delete'
    dfaux = dfaux.drop('racesWonByConstructor', axis = 1)
    df.loc[df['date'] == date, 'racesWonByConstructor'] = dfaux.merge(races_won_by_constructor, on=['date','constructorId'], how='left')['racesWonByConstructor'].values

    #Constructor times retired
    races_retired_by_constructor = df[df.date < date].groupby('constructorId')['positionText'].apply(lambda x: (x=='R').sum()).reset_index(name='racesRetiredByConstructor')
    races_retired_by_constructor['date'] = date
    dfaux = df[df['date'] == date].copy()
    dfaux['racesRetiredByConstructor'] = 'to_delete'
    dfaux = dfaux.drop('racesRetiredByConstructor', axis = 1)
    df.loc[df['date'] == date, 'racesRetiredByConstructor'] = dfaux.merge(races_retired_by_constructor, on=['date','constructorId'], how='left')['racesRetiredByConstructor'].values

100%|██████████| 1055/1055 [03:17<00:00,  5.35it/s]


**Format qualifying times**

In [16]:
qualifying['q1'] = qualifying['q1'].str.replace('.',':')
qualifying['q2'] = qualifying['q2'].str.replace('.',':')
qualifying['q3'] = qualifying['q3'].str.replace('.',':')
qualifying[['q1', 'q2', 'q3']] = qualifying[['q1', 'q2', 'q3']].applymap(lambda x: pd.to_timedelta(x, errors='coerce'))
qualifying['bestLap'] = qualifying[['q1','q2','q3']].min(axis=1)

**Calculate percent diference between drivers qualifying time and best time**

In [17]:
quali_aux = pd.merge(df[['raceId','driverId']], qualifying[['bestLap','raceId','driverId']], on = ['raceId','driverId'], how = 'left')
quali_aux_best = quali_aux.groupby(['raceId'], as_index=False)['bestLap'].min()
df = df.merge(quali_aux, on = ['raceId','driverId'], how = 'left')
df = df.merge(quali_aux_best, on = ['raceId'], how = 'left')
df['percentageOfBestQuali'] = (df['bestLap_x'] / df['bestLap_y'])*100
df = df.drop(columns=['bestLap_x', 'bestLap_y'])

**Save final dataframe**

In [18]:
df.to_csv('../data/processed/0.1-initial-exploration.csv')

In [19]:
races['name'].unique()

array(['Australian Grand Prix', 'Malaysian Grand Prix',
       'Chinese Grand Prix', 'Bahrain Grand Prix', 'Spanish Grand Prix',
       'Monaco Grand Prix', 'Turkish Grand Prix', 'British Grand Prix',
       'German Grand Prix', 'Hungarian Grand Prix', 'European Grand Prix',
       'Belgian Grand Prix', 'Italian Grand Prix', 'Singapore Grand Prix',
       'Japanese Grand Prix', 'Brazilian Grand Prix',
       'Abu Dhabi Grand Prix', 'Canadian Grand Prix', 'French Grand Prix',
       'United States Grand Prix', 'San Marino Grand Prix',
       'Austrian Grand Prix', 'Argentine Grand Prix',
       'Luxembourg Grand Prix', 'Portuguese Grand Prix',
       'Pacific Grand Prix', 'South African Grand Prix',
       'Mexican Grand Prix', 'Korean Grand Prix', 'Detroit Grand Prix',
       'Dutch Grand Prix', 'Dallas Grand Prix',
       'United States Grand Prix West', 'Swiss Grand Prix',
       'Caesars Palace Grand Prix', 'Swedish Grand Prix',
       'Indianapolis 500', 'Moroccan Grand Prix', 'Pes

In [20]:
drivers['nationality'].unique()

array(['British', 'German', 'Spanish', 'Finnish', 'Japanese', 'French',
       'Polish', 'Brazilian', 'Italian', 'Australian', 'Austrian',
       'American', 'Dutch', 'Colombian', 'Portuguese', 'Canadian',
       'Indian', 'Hungarian', 'Irish', 'Danish', 'Argentine', 'Czech',
       'Malaysian', 'Swiss', 'Belgian', 'Monegasque', 'Swedish',
       'Venezuelan', 'New Zealander', 'Chilean', 'Mexican',
       'South African', 'Liechtensteiner', 'Rhodesian',
       'American-Italian', 'Uruguayan', 'Argentine-Italian', 'Thai',
       'East German', 'Russian', 'Indonesian'], dtype=object)

In [21]:
df.columns

Index(['resultId', 'raceId', 'driverId', 'constructorId', 'gridStart',
       'position', 'positionText', 'positionOrder', 'pointsGained', 'laps',
       'time_x', 'milliseconds', 'fastestLap', 'rank', 'fastestLapTime',
       'fastestLapSpeed', 'statusId', 'driverRef', 'nationality', 'year',
       'round', 'circuitId', 'circuitName', 'date', 'ageAtRace', 'dateOfDebut',
       'ageAtDebut', 'yearsExperience', 'racingAtHome', 'driverStandingsId',
       'driverStandingsPoints', 'driverStandingsPosition',
       'driverStandingsWins', 'lastRaceRank', 'constructorStandingsId',
       'constructorStandingsPoints', 'constructorStandingsPosition',
       'constructorStandingsWins', 'previousRaceGridStart',
       'previousRacePosition', 'status', 'racesWon', 'racesRetired',
       'racesFinished', 'polePositions', 'racesWonByConstructor',
       'racesRetiredByConstructor', 'percentageOfBestQuali'],
      dtype='object')